## Introducción a PyTorch (Parte 7)

<a target="_blank" href="https://colab.research.google.com/github/pglez82/DeepLearningWeb/blob/master/labs/notebooks/Introducci%C3%B3n%20a%20PyTorch%20(Parte%207).ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
from datasets import load_dataset

# Cargar el dataset
dataset = load_dataset('sst')

train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

print(train_dataset)
print(val_dataset)
print(test_dataset)

print("Usando el dispositivo %s" % device)

### Procesado y visualización de datos
En este dataset la etiqueta es un número de real, que va de 0 a 1, según la opinión sea negativa (cercano a cero) o positiva (cercano a uno). Nosotros vamos a convertir este problema a un problema binario, considerando que las opiniones con más de 0.5, son positivas y viceversa.

In [ ]:
from torch.utils.data import Dataset
from transformers import BertTokenizer
import torch

# Creamos un dataset específico para nuestro problema
class SST2Dataset(Dataset):
    def __init__(self, sentences, labels, tokenizer):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.encodings = tokenizer(sentences, truncation=True, padding=True)
        self.labels = [1 if label>=0.5 else 0 for label in labels] 

    def __getitem__(self, idx):
        #Para un elemento dado devolvemos las tres cosas que nos devuelve el tokenizer: input_ids, token_type_ids y attention_masks
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        item['sentence'] = self.sentences[idx]
        return item

    def __len__(self):
        return len(self.labels)

base_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(base_model_name)
train_dataset = SST2Dataset(train_dataset['sentence'],train_dataset['label'], tokenizer=tokenizer)
val_dataset = SST2Dataset(val_dataset['sentence'],val_dataset['label'], tokenizer=tokenizer)

Como puedes ver en el código anterior, hemos creado un Dataset específico para procesar los datos de nuestro problema. En el constructor tokenizamos todas las frases del dataset. El tokenizador nos devuelve tres elementos, que posteriormente devolveremos cuando se nos pida un elemento del dataset:
- input_ids
- token_type_ids
- attention_masks
Además devolvemos:
- label: la etiqueta del ejemplo
- sentence: la frase original (no se necesita para el entrenamiento)

In [ ]:
import matplotlib.pyplot as plt

count_0 = train_dataset.labels.count(0)
count_1 = train_dataset.labels.count(1)

# Plotting
labels = ['Negativa', 'Positiva']
counts = [count_0, count_1]

plt.bar(labels, counts)
plt.title('Distribución de las opiniones en el dataset')
plt.show()



### Carga del modelo base y fine-tuning

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader


model = BertForSequenceClassification.from_pretrained(base_model_name)

# Crear dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
num_epochs = 5

for epoch in range(num_epochs):
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Example inference
test_text = "This movie is great!"
test_encoding = tokenizer.encode_plus(test_text, truncation=True, padding=True, return_tensors='pt')
test_input_ids = test_encoding['input_ids'].to(device)
test_attention_mask = test_encoding['attention_mask'].to(device)

model.eval()
with torch.no_grad():
    outputs = model(input_ids=test_input_ids, attention_mask=test_attention_mask)

predicted_probabilities = torch.softmax(outputs.logits, dim=1)
predicted_label = torch.argmax(predicted_probabilities, dim=1).item()

sentiment = "positive" if predicted_label == 1 else "negative"

print("Test text:", test_text)
print("Predicted sentiment:", sentiment)
print("Predicted probabilities:", predicted_probabilities)
